In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transactions = pd.read_csv("drive/MyDrive/Colab Notebooks/transactions.csv",
            sep = ',', nrows=1000000)
tr_types = pd.read_csv("drive/MyDrive/Colab Notebooks/tr_types.csv",
            sep = ';')
gender_train = pd.read_csv("drive/MyDrive/Colab Notebooks/gender_train.csv",
            sep = ',')
tr_mcc_codes = pd.read_csv("drive/MyDrive/Colab Notebooks/tr_mcc_codes.csv",
            sep = ';')

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L).

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:
transactions_sample = transactions.sample(n = 1000)

In [ ]:
count = transactions_sample[transactions_sample['tr_type'].isin(tr_types[tr_types['tr_description'].str.contains('POS|ATM')]['tr_type'])].shape[0]
fraction = count / transactions_sample.shape[0]

print(fraction)

0.395


## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
freq = transactions_sample['tr_type'].value_counts(normalize=True, sort=True)

top_10_tr = freq.head(10)
print(top_10_tr)

for tr_type in top_10_tr.index:
  print(tr_types[tr_types['tr_type'] == tr_type])
#print(tr_types[tr_types['tr_type'].isin(top_10_tr.index)][['tr_type', 'tr_description']].drop_duplicates())

1010    0.244
2010    0.155
1110    0.145
7070    0.142
1030    0.108
2370    0.060
7030    0.025
7010    0.016
1200    0.016
7071    0.014
Name: tr_type, dtype: float64
    tr_type         tr_description
98     1010  Покупка. POS ТУ СБ РФ
     tr_type                         tr_description
106     2010  Выдача наличных в АТМ Сбербанк России
     tr_type          tr_description
101     1110  Покупка. POS ТУ Россия
    tr_type                                     tr_description
59     7070  Перевод на карту (с карты) через Мобильный бан...
    tr_type                  tr_description
99     1030  Оплата услуги. Банкоматы СБ РФ
     tr_type                                     tr_description
129     2370  Списание с карты на карту по операции <перевод...
    tr_type                                     tr_description
47     7030  Перевод на карту (с карты) через АТМ (в предел...
    tr_type                                tr_description
39     7010  Взнос наличных через АТМ (в своем тер.банке

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
max_income_client = transactions[transactions.amount > 0].groupby('customer_id')['amount'].sum().sort_values(ascending = False).head(1)
print(max_income_client)

customer_id
70780820    1.248115e+09
Name: amount, dtype: float64


In [ ]:
max_expence_client = transactions[transactions.amount < 0].groupby('customer_id')['amount'].sum().sort_values().head(1)
print(max_expence_client)

customer_id
70780820   -1.249952e+09
Name: amount, dtype: float64


In [ ]:
diff = np.abs(max_income_client.max() + max_expence_client.max())

print(diff)

1837317.980000019


## Доп. задание

In [ ]:
print("Пол клиента: ")
print(gender_train[gender_train['customer_id'] == max_income_client.index[0]])


print("Категории, в которых он тратит деньги: ")
client_transactions = transactions[transactions['customer_id'] == max_income_client.index[0]]
client_transactions = client_transactions['tr_type']
client_transactions = client_transactions.unique()
for tr_type in client_transactions:
  print(tr_types[tr_types['tr_type'] == tr_type])


print("Клиент противоположного пола с ближайшими тратами")

opposite_sex_client = transactions[(transactions.amount < 0) & (transactions.gender == 0)].groupby('customer_id')['amount'].sum().to_frame()

target_expense = max_expence_client.to_frame().iloc[0][0]


opposite_sex_client = opposite_sex_client.loc[(opposite_sex_client['amount'] - target_expense).abs().idxmin()]
print(opposite_sex_client.name)

print("Категории в которых он тратит:")
client_transactions = transactions[transactions['customer_id'] == opposite_sex_client.name]
client_transactions = client_transactions['tr_type']
client_transactions = client_transactions.unique()
for tr_type in client_transactions:
  print(tr_types[tr_types['tr_type'] == tr_type])

Пол клиента: 
      customer_id  gender
8036     70780820       1
Категории, в которых он тратит деньги: 
     tr_type                         tr_description
106     2010  Выдача наличных в АТМ Сбербанк России
     tr_type                                     tr_description
118     2330  Списание с карты по операции “перевода с карты...
     tr_type                                     tr_description
129     2370  Списание с карты на карту по операции <перевод...
    tr_type                  tr_description
35     6110  Возврат покупки. POS ТУ Россия
    tr_type            tr_description
43     7020  Взнос наличных через POS
    tr_type                                     tr_description
59     7070  Перевод на карту (с карты) через Мобильный бан...
    tr_type                                     tr_description
47     7030  Перевод на карту (с карты) через АТМ (в предел...
    tr_type                                     tr_description
60     7071  Перевод на карту (с карты) через Мобильный

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
mean_top_10 = transactions[transactions['tr_type'].isin(top_10_tr.index)].groupby('tr_type').mean(['amount'])
print(mean_top_10)
#mean_top_10['mean_amount'] = mean_top_10.groupby('tr_type').mean(['amount'])
#print(mean_top_10)

median_top_10 = transactions[transactions['tr_type'].isin(top_10_tr.index)].groupby('tr_type').median(['amount'])
print(median_top_10)

          customer_id     mcc_code         amount    gender    tr_hour
tr_type                                                               
1010     5.039681e+07  5613.395224  -19784.748640  0.490903  13.821186
1030     4.989148e+07  4814.007144   -5320.980222  0.420001  12.767128
1110     5.081484e+07  5595.733586  -32119.330371  0.534669   5.537034
1200     5.130010e+07  6299.572704  -35440.107159  0.616155   0.532105
2010     5.019928e+07  6011.000000 -136077.629325  0.493195  12.666863
2370     5.137999e+07  4829.000000 -205418.249032  0.446197  13.306121
7010     5.120879e+07  6011.000000  276391.789596  0.471838  13.173892
7030     6.209183e+07  6010.000000   86104.332909  0.719004  10.592070
7070     6.408689e+07  6010.000000   65569.831700  0.795462  11.322309
7071     4.782654e+07  6010.000000   66806.826623  0.843147  13.225081
         customer_id  mcc_code     amount  gender  tr_hour
tr_type                                                   
1010      49724677.0    5499.0

In [ ]:
mean_client = transactions[transactions['customer_id'].isin(max_income_client.index)]['amount'].mean()
print(mean_top_10)

median_client = transactions[transactions['customer_id'].isin(max_income_client.index)]['amount'].median()
print(median_top_10)

-194731.06740639365
-67377.47


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, on='customer_id', how='left')
transactions = pd.merge(transactions, tr_mcc_codes, on='mcc_code', how='inner')
transactions = pd.merge(transactions, tr_types, on='tr_type', how='inner')


transactions.shape

(999584, 9)

In [ ]:
print(transactions.head(n = 10))
print(transactions.shape)

   customer_id  tr_datetime  mcc_code  tr_type   amount term_id  gender  \
0     39026145   0 10:23:26      4814     1030 -2245.92     NaN     1.0   
1     39026145   6 07:08:31      4814     1030 -5614.79     NaN     1.0   
2     39026145   8 07:06:10      4814     1030 -1122.96     NaN     1.0   
3     39026145  11 08:49:03      4814     1030 -2245.92     NaN     1.0   
4     39026145  11 14:12:08      4814     1030 -2245.92     NaN     1.0   
5     39026145  11 14:15:30      4814     1030 -2245.92     NaN     1.0   
6     39026145  13 11:17:34      4814     1030 -2245.92     NaN     1.0   
7     39026145  18 07:39:05      4814     1030 -2245.92     NaN     1.0   
8     39026145  19 13:24:35      4814     1030  -449.18     NaN     1.0   
9     39026145  19 13:25:31      4814     1030 -1122.96     NaN     1.0   

                                     mcc_description  \
0  Звонки с использованием телефонов, считывающих...   
1  Звонки с использованием телефонов, считывающих...   
2  Зво

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
mean_spent_diff = abs(transactions[(transactions['amount'] < 0) & (transactions['gender'] == 0)]['amount'].mean() - transactions[(transactions['amount'] < 0) & (transactions['gender'] == 1)]['amount'].mean())
print(mean_spent_diff)

32718.05492022467


In [ ]:
mean_income_diff = abs(transactions[(transactions['amount'] > 0) & (transactions['gender'] == 0)]['amount'].mean() - transactions[(transactions['amount'] > 0) & (transactions['gender'] == 1)]['amount'].mean())
print(mean_income_diff)

63366.57104801465


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
max_income_by_type = transactions[transactions['amount'] > 0].groupby(['gender', 'tr_type'])['amount'].agg('max').reset_index()

female_least10 = max_income_by_type[max_income_by_type['gender'] == 0].sort_values('amount').head(10)
male_least10 = max_income_by_type[max_income_by_type['gender'] == 1].sort_values('amount').head(10)

print(female_least10)
print(male_least10)

    gender  tr_type    amount
10     0.0     4051   1122.96
12     0.0     4110   2245.92
13     0.0     4210   2245.92
9      0.0     2370   2335.75
11     0.0     4100   5041.86
36     0.0     7075   6737.75
6      0.0     2110  22459.16
0      0.0     1010  24839.83
5      0.0     2010  44918.32
7      0.0     2210  68315.82
    gender  tr_type     amount
47     1.0     4110    2245.92
44     1.0     2020    2245.92
38     1.0     1010   28803.87
71     1.0     8100   64682.37
48     1.0     6000   79280.83
66     1.0     7041   87590.72
43     1.0     2010   89836.63
45     1.0     2110  112295.79
46     1.0     2370  113194.15
57     1.0     7015  121279.45


In [ ]:
common_tr_types = pd.merge(female_least10, male_least10, on='tr_type', how='inner')['tr_type']
print(common_tr_types)

0    4110
1    2370
2    2110
3    1010
4    2010
Name: tr_type, dtype: int64


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
male_spent_by_mcc = transactions[(transactions['gender'] == 0) & (transactions['amount'] < 0)].groupby('mcc_code')['amount'].sum()
female_spent_by_mcc = transactions[(transactions['gender'] == 1) & (transactions['amount'] < 0)].groupby('mcc_code')['amount'].sum()

print(male_spent_by_mcc)
print(female_spent_by_mcc)

mcc_code
742      -386207.68
1711     -674897.69
1799     -261626.73
2741      -20746.42
3000   -74431645.27
           ...     
8699     -521107.48
8999   -19432057.92
9222     -361143.24
9311     -406197.45
9399    -1481389.65
Name: amount, Length: 174, dtype: float64
mcc_code
742      -132277.93
1711     -206264.90
1731     -121542.67
1799    -1129257.68
2741     -221290.08
           ...     
8999   -28643346.63
9222     -988205.17
9311     -373271.22
9399    -1810820.67
9402       -4581.67
Name: amount, Length: 175, dtype: float64


In [ ]:
spent_by_mcc = pd.concat([male_spent_by_mcc, female_spent_by_mcc], axis=1)
spent_by_mcc.columns = ['male', 'female']
#####print(spent_by_mcc)

spent_by_mcc['diff'] = spent_by_mcc['male'] - spent_by_mcc['female']

top_10_diff_mcc = spent_by_mcc.reindex(spent_by_mcc['diff'].abs().sort_values(ascending=False).index)[:10]

# top_10_diff_mcc['mcc_description'] = top_10_diff_mcc.index.map(tr_mcc_codes)
top_10_diff_mcc = pd.merge(top_10_diff_mcc, tr_mcc_codes, left_on='mcc_code', right_on='mcc_code', how='left')
print(top_10_diff_mcc)
# print(tr_types[tr_types['tr_type'].isin(top_10_diff_mcc.index)][['tr_type', 'tr_description']].drop_duplicates())

   mcc_code          male        female          diff  \
0      6011 -4.958077e+09 -8.043939e+09  3.085862e+09   
1      4829 -3.513864e+09 -6.013017e+09  2.499153e+09   
2      6010 -2.612731e+08 -4.821311e+08  2.208580e+08   
3      5541 -5.162489e+07 -2.070574e+08  1.554326e+08   
4      5511 -5.095159e+06 -7.862393e+07  7.352877e+07   
5      5812 -5.787961e+07 -1.282582e+08  7.037856e+07   
6      5977 -7.360784e+07 -3.119022e+07 -4.241762e+07   
7      5533 -1.278980e+07 -5.512787e+07  4.233808e+07   
8      7995 -1.189100e+05 -4.091455e+07  4.079564e+07   
9      5661 -8.183139e+07 -4.367668e+07 -3.815470e+07   

                                     mcc_description  
0  Финансовые институты — снятие наличности автом...  
1                                  Денежные переводы  
2   Финансовые институты — снятие наличности вручную  
3                            Станции техобслуживания  
4  Легковой и грузовой транспорт — продажа, серви...  
5             Места общественного питания,

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.

In [ ]:
transactions['tr_hour'] = transactions['tr_datetime'].str.split(':').str[0].str[-2:].astype(int)

night_transactions = transactions.query('amount < 0 and tr_hour < 6').groupby(['gender', 'tr_hour']).size()
print(night_transactions)
night_transactions = night_transactions.groupby(['gender']).sum()
print(night_transactions)

#print("Число ночных транзакций:" + np.sum(night_transactions, axis=-1))

gender  tr_hour
0.0     0          29060
        1            917
        2           1289
        3           1893
        4           2252
        5           3416
1.0     0          35697
        1            984
        2            921
        3           1191
        4           1615
        5           2609
dtype: int64
gender
0.0    38827
1.0    43017
dtype: int64
